# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [53]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [88]:
# set the parameters to run the script
directories = ['/media/elea/ssd/sdgasf']
calib_directory = '/media/elea/ssd/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'both'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600, 650]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'no_viz'
save_pdf_figs = False


parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs)

In [89]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: ['/media/elea/ssd/sdgasf/2024-09-26_T_CHB-2_FX_A1_1']


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.36s/it]


processing without PDDN done.
processing with PDDN...


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Processing: ['/media/elea/ssd/sdgasf/2024-09-26_T_CHB-2_FX_A1_1']
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_550_Fresh_HB.pt -- Complete
*** PDDN DENOISING FILTER *** 
 >> PDDN Denoising: total time-points: 1000
 >> PDDN Denoising: start time-point: 1
 >> PDDN Denoising: final time-point: 0
 >> PDDN Denoising: recursive filtering enabled (2 x cascade): False
 >> MPI Denoising PDDN Performance: Elapsed time = 1.372 s
 
 >> Exported X3D .cod file as: ./temp_processing/2024-09-26_T_CHB-2_FX_A1_1/raw_data/550nm/550_Intensite_PDDN.cod
 >> PDDN Loading Model: /home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/PDDN_model/PDDN_model_600_Fresh_HB.pt -- ...
 [wrn] PDDN Model Summary NOT Available!
 >> PDDN Loading Model: /home/e

100%|█████████████████████████████████████████████| 1/1 [00:04<00:00,  4.73s/it]

processing with PDDN done.
CPU times: user 15.5 s, sys: 1.23 s, total: 16.7 s
Wall time: 6.1 s


{'MM_processing': 0.3247041702270508,
 'azimuth_std_processing': 0.006949424743652344,
 'denoising': 0.3100588321685791,
 'save_npz': 0.1214146614074707,
 'full_processing': 2.176048755645752,
 'plotting': 7.152557373046875e-07,
 'total': 4.729862213134766}

In [90]:
%%time

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = batch_processing.batch_visualization(parameters)
times

CPU times: user 16.1 s, sys: 1.55 s, total: 17.7 s
Wall time: 15.3 s


15.30210828781128

In [92]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all)

Aligning wavelenght: 600nm


  0%|                                                     | 0/1 [00:00<?, ?it/s]

 
 >> Exported X3D .cod file as: /media/elea/ssd/sdgasf/2024-09-26_T_CHB-2_FX_A1_1/raw_data/600nm/600_Intensite_aligned.cod
 
 >> Exported X3D .cod file as: /media/elea/ssd/sdgasf/2024-09-26_T_CHB-2_FX_A1_1/raw_data/600nm/600_Intensite_PDDN_aligned.cod


100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.05s/it]


Aligning wavelenght: 650nm


100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2562.19it/s]


In [94]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Chris
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

  0%|                                                     | 0/1 [00:00<?, ?it/s]

{'processed': True, 'calibration_directories': '/media/elea/ssd/calib/2024-09-18_C_1', 'parameters': {'directories': ['/media/elea/ssd/sdgasf'], 'calib_directory': '/media/elea/ssd/calib', 'wavelengths': [550, 600, 650], 'parameter_set': 'default', 'PDDN': 'both', 'run_all': True, 'processing_mode': 'no_viz', 'time_mode': True, 'temp_folder': './temp_processing', 'save_pdf_figs': False}, 'libmpMuelMat': '1.0', 'processingmm': '1.2.0'}


AssertionError: Calibration directory does not exist